# Modelagem SEIR
Neste notebook está implementado o modelo SEIR de Zhilan Feng. Que inclui quarentena, e hospitalizações

\begin{align}
\frac{dS}{dt}&=-\beta S (I+(1-\rho)H)\\
\frac{dE}{dt}&= \beta S (I+(1-\rho)H)-(\chi+\alpha)E\\
\frac{dQ}{dt}&=\chi E -\alpha Q\\
\frac{dI}{dt}&= \alpha E - (\phi+\delta)I\\
\frac{dH}{dt}&= \alpha Q +\phi I -\delta H\\
\frac{dR}{dt}&= \delta I +\delta H
\end{align}

Denotando o tamanho total da epidemia por $Y_e(t)=E(t)+Q(t)+I(t)+H(t)$, podemos escrever
\begin{align}
\nonumber \frac{dY_e}{dt}&= \frac{dE}{dt}+\frac{dQ}{dt}+\frac{dI}{dt}+\frac{dH}{dt}\\
\label{ye}&=\beta S (I(1-\rho)H) -\delta(I+H)
\end{align}


In [13]:
from scipy.integrate import odeint
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['beta', 'interactive']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
def seqihr(y,t,*params):
    S,E,Q,I,H,R = y
    chi,phi,beta,rho,delta,alpha = params
    return[
        -beta*S*(I+(1-rho)*H), #dS/dt
        beta*S*(I+(1-rho)*H) - (chi+alpha)*E,#dE/dt
        chi*E -alpha*Q,#dQ/dt
        alpha*E - (phi+delta)*I,#dI/dt
        alpha*Q + phi*I -delta*H,#dH/dt
        delta*I + delta*H,#dR/dt
    ]

In [15]:
chi=.05  # Taxa de quarentenação
phi=.01 # Taxa de Hospitalização
beta=.2 #taxa de Transmissão
rho=.6 # Attenuation of the transmission when in hospital
delta=1/10 # Taxa de recuperação hospitalar
alpha=1/3 # Taxa de incubaçao

In [16]:
inits = [0.9,.1,0,0,0,0]
trange = arange(0,100,.1)
res = odeint(seqihr,inits,trange,args=(chi,phi,beta,rho,delta,alpha))

In [17]:

@interact(chi=(0,.1, 0.01),phi=(0,.1,.005),beta=(0,.5,.05),rho=(0,1,.1),delta=(0,1,.05),alpha=(0,1,0.01))
def plota_simulação(chi=0.05,phi=.01,beta=.2,rho=.6,delta=.1,alpha=.33):
    res = odeint(seqihr,inits,trange,args=(chi,phi,beta,rho,delta,alpha))
    fig, ax = subplots(1,1, figsize=(15,10))
    ax.plot(trange,res[:,1:-1])
    ax.set_ylabel('Fração da População')
    ax.set_xlabel('Tempo (dias)')
    ax.grid()
    ax.legend(['Exposto','Quar','Inf','Hosp']);

interactive(children=(FloatSlider(value=0.05, description='chi', max=0.1, step=0.01), FloatSlider(value=0.01, …

## Adicionando Assintomáticos
![Diagrama de blocos](seqiahr.jpg "Modelo")
O modelo de Feng não é apropriado para representar a epidemia da COVID-19. Vamos Modificá-lo para incluir entre outras coisas, os assintomáticos. Para simplificar a notação vamos denotar por $\lambda=\beta(I\color{red}{+A}+(1-\rho)H)$, já incluindo os assintomáticos, $A$.

\begin{align}
\frac{dS}{dt}&=-\lambda S \\
\frac{dE}{dt}&= \lambda S -(\chi+\alpha)E\\
\frac{dQ}{dt}&=\chi E -\color{red}{(\phi+\delta)} Q\\
\frac{dI}{dt}&= \color{red}{(1-p)}\alpha E - (\phi+\delta)I\\
\color{red}{\frac{dA}{dt}}&= \color{red}{p\alpha E -\delta A}\\
\frac{dH}{dt}&= \color{red}{\phi} Q +\phi I -\delta H\\
\frac{dR}{dt}&= \delta I +\delta H \color{red}{+ \delta A +\delta Q}
\end{align}

Neste novo modelo alteramos mais algumas coisinhas resaltadas em vermelho acima. em primeiro lugar fizemos com que a taxa de hospitalização dos quarentenados, seja a mesma que a dos infectados e permitimos que se recuperem sem hospitalização. Além disso, para alimentar a classe dos Assintomáticos, definimos $p$ como a fração de assintomáticos na população. $R_0\approx$

In [23]:
def seqiahr(y,t,*params):
    S,E,Q,I,A,H,R = y
    chi,phi,beta,rho,delta,alpha,p = params
    return[
        -beta*S*(I+A+(1-rho)*H), #dS/dt
        beta*S*(I+A+(1-rho)*H) - (chi+alpha)*E,#dE/dt
        chi*E -(phi+delta)*Q,#dQ/dt
        (1-p)*alpha*E - (phi+delta)*I,#dI/dt
        p*alpha*E - delta*A,
        phi*Q + phi*I -delta*H,#dH/dt
        delta*I + delta*H+delta*A + delta*Q,#dR/dt
    ]

In [48]:
inits = [0.9,.1,0, 0,0,0,0]
@interact(chi=(0,1, 0.05),phi=(0,.5,.01),beta=(0,1.5,.05),rho=(0,1,.1),delta=(0,1,.05),alpha=(0,1,0.01),p=(0,1,.05))
def plota_simulação(chi=0.05,phi=.01,beta=.25,rho=.6,delta=.1,alpha=.33, p=.5):
    res = odeint(seqiahr,inits,trange,args=(chi,phi,beta,rho,delta,alpha,p))
    et = delta/beta
    idx = np.argwhere(np.abs(res[:,0]-et)==min(np.abs(res[:,0]-et)))[0,0]
    fig, ax = subplots(1,1, figsize=(15,10))
    ax.plot(trange,res[:,1:-1])
    ax.text(80,0.08,f"$R_0~{beta/delta}$\n$p={p}$\n$\\rho={rho}$")
    ax.vlines(trange[idx],0,res[:,1:-1].max())
    ax.set_ylabel('Fração da População')
    ax.set_xlabel('Tempo (dias)')
    ax.grid()
    ax.legend(['Exposto','Quar','Inf', 'Assintomático','Hosp']);

interactive(children=(FloatSlider(value=0.05, description='chi', max=1.0, step=0.05), FloatSlider(value=0.01, …